<a href="https://colab.research.google.com/github/maysahayu/Collab-Project/blob/main/scrap_prepro_fe_cluster_Webtoon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import csv
import os
hades = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}

In [19]:
# SCRAPING DATA

def scraping_springer():
    data_list = []
    base_url = "https://www.webtoons.com/id/"
    headers = {'User-Agent': 'Mozilla/5.0'}
    genres = ["drama", "fantasy", "romantic_fantasy", "comedy", "action", "slice_of_life", "romance", "thriller", "horror", "local"]

    for page in genres:
        url = f"https://www.webtoons.com/id/genres/{page}?sortOrder=READ_COUNT"

        try:
            halaman_utama = req.get(url, headers=headers).text
            soup = bs(halaman_utama, 'lxml')

            comics = soup.select('ul.card_lst a.card_item')

            if not comics:
                print(f"Tidak ada artikel ditemukan di halaman {page}.")
                continue

            for comic in comics:
                judul_element = comic.find('p', class_='subj')
                judul = judul_element.text.strip() if judul_element else 'Judul tidak ditemukan'

                author_element = comic.find('p', class_='author')
                author = author_element.text.strip() if author_element else 'Author tidak ditemukan'

                # URL halaman detail komik
                detail_url = comic['href']

                # Masuk ke halaman detail untuk mengambil ringkasan
                try:
                    detail_page = req.get(detail_url, headers=headers).text
                    detail_soup = bs(detail_page, 'lxml')

                    # Cari elemen ringkasan di halaman detail
                    desc_element = detail_soup.find('p', class_='summary')
                    desc = desc_element.text.strip() if desc_element else 'Ringkasan tidak ditemukan'

                except Exception as e:
                    print(f"Terjadi kesalahan saat mengakses {detail_url}: {e}")
                    desc = 'Ringkasan tidak ditemukan'  # Menangani jika halaman detail gagal diakses

                # Menambahkan data komik yang berhasil di-scrape
                data_list.append([judul, author, desc])

        except Exception as e:
            print(f"Terjadi kesalahan saat mengakses {url}: {e}")

    # Menampilkan hasil scraping dalam bentuk DataFrame
    df = pd.DataFrame(data_list, columns=['Judul Komik', 'Author', 'Ringkasan'])
    print(df)

    # Menyimpan DataFrame ke dalam file CSV
    df.to_csv('data_webtoon.csv', index=False, encoding='utf-8')

    print("Data berhasil disimpan")


In [20]:
scraping_springer()

                     Judul Komik                     Author  \
0                        LOOKISM               Taejoon Park   
1                   Girl's World                    morangg   
2                The Real Lesson  CHAE YONGTAEK / HAN GARAM   
3     Undercover at Chaebol High                         AJ   
4               GOOD/BAD FORTUNE               Ariel Duyung   
...                          ...                        ...   
1499      SCRAMBLED: Journeylism                Lintankleen   
1500                Dracko Diary                   Indra AD   
1501                Perhaps Mine         Cikakey / Candrasa   
1502                 Summer Rain                  kiyoshin_   
1503                  Oh My Boy!                     Kanipa   

                                              Ringkasan  
0        Cowok ini punya 2 wujud! Pilih yang mana ya..?  
1      Jadi bebek di antara para angsa? We will surive!  
2     Setelah undang-undang larangan memukul para si...  
3     Lee D

In [24]:
# PRE PROCESSING DATA

import pandas as pd
import string
import re
import emoji
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords')

# File paths
file_path = "data_webtoon.csv"
output_path = "prepro_webtoon.csv"
kamus_alay = "colloquial-indonesian-lexicon.csv"

kolom_target = 'Ringkasan'  # Kolom target yang akan diproses

# Mendapatkan daftar stopwords
stop_words = set(stopwords.words('indonesian'))

# Fungsi untuk menghilangkan tanda baca
def remove_punctuation(text):
    if isinstance(text, str):
        return text.translate(str.maketrans('', '', string.punctuation))
    return text

# Fungsi untuk menghilangkan URL dari teks
def remove_url(text):
    if isinstance(text, str):
        url_pattern = r'http\S+|www\S+'
        return re.sub(url_pattern, '', text)
    return text

# Fungsi untuk menghilangkan HTML tags dari teks
def remove_html_tags(text):
    if isinstance(text, str):
        return re.sub(r'<.*?>', '', text)  # Menghapus semua yang ada di antara <>
    return text

# Fungsi untuk menghapus stopwords
def remove_stopwords(text):
    if isinstance(text, str):
        words = re.findall(r'\b\w+\b', text)  # Mengambil kata-kata yang valid
        return ' '.join([word for word in words if word not in stop_words])
    return text

def remove_digits(text):
    if isinstance(text, str):
        return re.sub(r'\d+', '', text)  # Menghapus semua digit
    return text

# Fungsi untuk mengonversi emoji menjadi teks deskripsi sederhana
def convert_emoji_to_text(text):
    if isinstance(text, str):
        return emoji.demojize(text)  # Mengonversi emoji menjadi teks
    return text

# Fungsi untuk menghilangkan simbol yang tidak diinginkan
def remove_special_characters(text):
    if isinstance(text, str):
        return text.replace("ð", "").replace("â", "").replace("º", "").replace("ï", "")
    return text

# Membaca file Kamus Alay dan membuat dictionary alay -> formal
df_slang = pd.read_csv(kamus_alay)
alay_dict = dict(zip(df_slang['slang'], df_slang['formal']))

# Fungsi untuk mengganti kata alay dengan kata formal
def normalize_alay(text):
    if isinstance(text, str):
        return ' '.join([alay_dict.get(word, word) for word in text.split()])
    return text

# Membaca file CSV
data = pd.read_csv(file_path)

# Mengubah teks menjadi lowercase pada kolom target
data[kolom_target] = data[kolom_target].str.lower()

# Menghapus URL terlebih dahulu
data[kolom_target] = data[kolom_target].apply(remove_url)

# Menghapus HTML tags setelah URL dihilangkan
data[kolom_target] = data[kolom_target].apply(remove_html_tags)

# Menghapus tanda baca setelah URL dihilangkan
data[kolom_target] = data[kolom_target].apply(remove_punctuation)

# Mengonversi emoji menjadi deskripsi teks sederhana
data[kolom_target] = data[kolom_target].apply(convert_emoji_to_text)

# Menghilangkan simbol yang tidak diinginkan dari teks
data[kolom_target] = data[kolom_target].apply(remove_special_characters)

# Menghapus digit angka
data[kolom_target] = data[kolom_target].apply(remove_digits)

# Menghapus stopwords
data[kolom_target] = data[kolom_target].apply(remove_stopwords)

# Mengganti kata-kata alay dengan kata formal
data[kolom_target] = data[kolom_target].apply(normalize_alay)

# Menyimpan hasilnya kembali ke file CSV
data.to_csv(output_path, index=False)

print(f"Semua teks telah diubah ke huruf kecil, URL, tanda baca, emoji telah dikonversi, stopwords, dan kata alay pada kolom '{kolom_target}' telah dihapus/diganti, dan file disimpan sebagai '{output_path}'")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Semua teks telah diubah ke huruf kecil, URL, tanda baca, emoji telah dikonversi, stopwords, dan kata alay pada kolom 'Ringkasan' telah dihapus/diganti, dan file disimpan sebagai 'prepro_webtoon.csv'


In [25]:
# FEATURE ENGINEERING (BOW)

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

cleaned_file_path = "prepro_webtoon.csv"
output_bow_path = "fe_webtoon.csv"

df_cleaned = pd.read_csv(cleaned_file_path)

df_cleaned['Ringkasan'] = df_cleaned['Ringkasan'].fillna('')

df_cleaned['Ringkasan'] = df_cleaned['Ringkasan'].astype(str)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_cleaned['Ringkasan'])

bow_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())


bow_df.to_csv(output_bow_path, index=False)

print(f"Hasil Bag of Words telah disimpan ke dalam file '{output_bow_path}'")

print(bow_df.head())

Hasil Bag of Words telah disimpan ke dalam file 'fe_webtoon.csv'
   aaron  abad  abadi  abang  abi  absurd  acara  acaraacara  achelans  \
0      0     0      0      0    0       0      0           0         0   
1      0     0      0      0    0       0      0           0         0   
2      0     0      0      0    0       0      0           0         0   
3      0     0      0      0    0       0      0           0         0   
4      0     0      0      0    0       0      0           0         0   

   adakah  ...  yusin  zahra  zaman  zan  zara  zener  zenus  zeron  zion  \
0       0  ...      0      0      0    0     0      0      0      0     0   
1       0  ...      0      0      0    0     0      0      0      0     0   
2       0  ...      0      0      0    0     0      0      0      0     0   
3       0  ...      0      0      0    0     0      0      0      0     0   
4       0  ...      0      0      0    0     0      0      0      0     0   

   zombie  
0       0  
1  